In [2]:
def read_alias(path):
    dict = {}
    data = None
    with open(path, 'r') as file:
        data = json.load(file)
        for product in data:
            for alias in data[product]:
                dict[alias.upper()] = product.upper()
    return dict

def read_productType_alias():
    return {
        "spuit" : "injectiespuit",
        "injectie" : "injectiespuit",
        "Drank" : "drank" #is makkelijk om de uppercase hier al te fixen
    }

#skeere oplossing, geen zin om een bestand aan te maken
#data integriteit is voor losers
def read_pricechanges():
    data = {
        118: {
            "oldname": "PRIADEL",
           "searchname": "PRIADEL 19",
            "date": "2019-07-01"
        }
    }
    return data
            
def get_cleaned_name(name, dict):
    name = name.upper()
    if name in dict:
        return dict[name].upper()
    else:
        return name

def set_product_searchnames(df, dict):
    df["SearchName"] = df["ProductName"].str.upper()
    for product in dict.items():
        id = product[0]
        searchname = product[1]["searchname"]
        df.loc[df['ProductID'] == id, 'SearchName'] = searchname
        print(searchname)
    return df

def change_priadel_price(cursor):
    sql = "update ProductInfo set SalesPrice = 4.67 where ProductID = 117"
    cursor.execute(sql)

In [4]:
def create_db(name : str, cursor):
    CREATE_ORDER_TABLE = """
    CREATE TABLE "OrderLines" (
	"OrderlineID"	INTEGER,
	"OrderID"	INTEGER,
	"CustomerID"	NUMERIC,
	"Date"	TEXT,
	"Product"	TEXT,
    "ProductName"	TEXT,
	"Amount"	REAL
    );
    """
    
    CREATE_PRODUCT_TABLE = """
    CREATE TABLE "ProductInfo" (
    	"ProductID"	INTEGER,
    	"ProductName"	TEXT,
        "SearchName" TEXT,
    	"ProductType"	TEXT,
    	"CostPrice"	REAL,
    	"SalesPrice"	REAL
    );
    """
    cursor.execute(CREATE_ORDER_TABLE)
    cursor.execute(CREATE_PRODUCT_TABLE)
    print("db created")

def update_product_types(cursor, oldtype, newtype):
    sql = """ 
        UPDATE ProductInfo 
        SET ProductType = ? 
        WHERE ProductType = ?
    """
    cursor.execute(sql, (newtype, oldtype))

def update_order_searchnames(cursor, searchname, newname, date):
    '''
    function for updating the name after the order has already been inserted
    only used for setting the priadel
    '''
    sql = """update OrderLines 
                set ProductName = ? 
            where date(Date) > date(?) 
            and ProductName = ?
          """
    cursor.execute(sql, (newname, date, searchname))

def insert_products_db(dataframe, cursor):
    INSERT_PRODUCT_SQL = "INSERT INTO ProductInfo (ProductID,ProductName,Searchname, ProductType, CostPrice, SalesPrice) VALUES (?,?,?,?,?,?)"
    for p in dataframe:
        cursor.execute(INSERT_PRODUCT_SQL, (p["ProductID"],p["ProductName"],p["SearchName"], p["ProductType"],p["CostPrice"],p["SalesPrice"]))
    
    
def insert_orders_db(dataframe, names : dict, cursor):
    INSERT_ORDER_LINE = """ INSERT INTO OrderLines (OrderlineID, OrderID, CustomerID, Date, Product, ProductName, Amount) VALUES (?,?,?,?,?,?,?)"""
    for line in dataframe:
        ProductName = get_cleaned_name(line["Product"], names)
        cursor.execute(INSERT_ORDER_LINE, (line["OrderlineID"], line["OrderID"], line["CustomerID"], line["Date"], line["Product"], ProductName, line["Amount"]))


In [5]:
def update_price_db(id : int, price: float, cursor):
    sql = "UPDATE ProductInfo set SalesPrice = ? WHERE ProductID = ?"
    cursor.execute(sql, (price, id))

In [6]:
import sqlite3
import json
import pandas as pd
import os

DB_NAME = "test.db"

orderlines = pd.read_csv("OrderLines.csv").to_dict(orient='records')
names = read_alias("known_alias.json")

conn = None
cursor = None

if os.path.isfile(DB_NAME):
    print("db bestaat al!")
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
else:
    #maak een nieuwe database aan met de tabellen
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    create_db(DB_NAME, cursor)
    #sla op
    conn.commit()
    print("db created")

#de orderlines inserten
#deze functie normaliseerd de ProductNames
print("inserting orders...")
insert_orders_db(orderlines, names, cursor)
#opslaan
conn.commit()
print("orders inserted")

print("inserting products...")
products = pd.read_csv("ProductInfo.csv", delimiter=";")#.to_dict(orient='records')
name_changes = read_pricechanges()
products = set_product_searchnames(products, name_changes)
products = products.to_dict(orient='records')
insert_products_db(products, cursor)
conn.commit()
print("products inserted")

print("updating priadel price...")
change_priadel_price(cursor)
conn.commit()
print("priadel price updated")

print("updating priadel orders...")
for item in name_changes.items():
    date = item[1]["date"]
    newname = item[1]["searchname"]
    oldname = item[1]["oldname"]
    update_order_searchnames(cursor,oldname, newname, date)
conn.commit()
print("done updating priadel orders")

print("updating product types...")
producttypes = read_productType_alias()
for old, new in producttypes.items():
    update_product_types(cursor, old, new)
conn.commit()
print("product types updated")

#db connectie sluiten
conn.close()

db created
db created
inserting orders...
orders inserted
inserting products...
PRIADEL 19
products inserted
updating priadel price...
priadel price updated
updating priadel orders...
done updating priadel orders
updating product types...
product types updated
